<a href="https://colab.research.google.com/github/DanielHolzwart/Scraping-stock-data-from-macrotrends-and-multplt/blob/main/Scraping_stock_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
import requests
from lxml import html as HTMLParser

In [6]:
#usingbeatifulsoup for html extraction
from bs4 import BeautifulSoup

In [13]:
#multpl is a nice website for S&P500 charts, having many key figures available. For this we are interested in the chart and P/E ratio
url = ['https://www.multpl.com/s-p-500-pe-ratio/table/by-month', 'http://www.multpl.com/s-p-500-historical-prices/table/by-month']

#define a function to extract data from an URL
def get_data_from_url(url):
    response = requests.get(url)
    response.raise_for_status()

    #get html ectract
    soup = BeautifulSoup(response.content, 'html.parser')

    #find tables for cart extraction
    table = soup.find('table')

    #the html file of multpl.com has special characters (a cross) for forcasted values, which we need to remove
    for abbr in table.find_all('abbr'):
        abbr.decompose()

    # Initialize lists to store the extracted data
    dates = []
    values = []

    # Iterate over the table rows
    for row in table.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')
        date = columns[0].get_text(strip=True)
        value = columns[1].get_text(strip=True)
        dates.append(date)
        values.append(value)

    return zip(dates,values)

In [14]:
#get the data for the P/E ratios and S&P500 chart
pe_values = get_data_from_url(url[0])
sp_values = get_data_from_url(url[1])

In [15]:
for i, x in enumerate(pe_values):
    print(x)
    if i == 2:
      break

('Nov 15, 2024', '30.00')
('Oct 1, 2024', '29.38')
('Sep 1, 2024', '28.69')


In [16]:
df_pe = pd.DataFrame(pe_values, columns=['Date', 'PE'])
df_sp = pd.DataFrame(sp_values, columns=['Date', 'Price'])

In [60]:
df = pd.concat([df_pe, df_sp.loc[:,'Price']], axis=1)

In [57]:
#after concatenating we quickly check for na values.
nans = 0
#get indices of NA values and store in dictionary
nans_indices = {}
for x in df.columns.tolist():
    nans += sum(df.loc[:,x].isna())
    indices = df[df[x].isna()].index.tolist()
    if indices:
        nans_indices[x] = indices
print(nans_indices)

{'Date': [1844, 1845, 1846], 'PE': [1844, 1845, 1846]}


In [56]:
#these old missing values are not going to be a problem for us

{'Date': [1844, 1845, 1846], 'PE': [1844, 1845, 1846]}

In [61]:
# parse date formats
df.Date = pd.to_datetime(df.Date, format='%b %d, %Y')
# transform to numeric values
df.PE = pd.to_numeric(df.PE)
df.Price = pd.to_numeric(df.Price.str.replace(',', '').astype(float))  # handle commas inside strings
df = df.set_index('Date')
df.head()

,PE,Price
Date,,
2024-08-01,27.96,5878.30
2024-07-01,28.27,5756.33
2024-06-01,27.64,5621.26
2024-05-01,26.93,5478.21
2024-04-01,26.50,5538.00


Now we are going to plot the graphgs. For this, we are going to cufflinks which is a very convenient way of plot panda series'

In [62]:
import numpy as np

In [63]:
import cufflinks as cf
from plotly.offline import iplot, init_notebook_mode
import plotly.express as px
import plotly.offline as pyo

cf.go_offline()

In [64]:
#the is needed as otherwise the graphs are just going to be empty spaces
import plotly.io as pio
pio.renderers.default = "colab"

In [67]:
#new dataframe only considering data from 2000 onwards
df1 = df.loc[df.index.year >= 2000]

In [68]:
fig = df1[['Price', 'PE']].iplot(
    subplots=True,
    shape=(2, 1),
    shared_xaxes=True,
    title="S&P500 time series with PE",
    asFigure=True
)

# Update layout
fig.update_layout(width=1000, height=600)  # Set figure size here

# Show the plot
iplot(fig)

/usr/local/lib/python3.10/dist-packages/cufflinks/plotlytools.py:117: FutureWarning:

DatetimeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.



In [70]:
#in my opining a stock chart without log values if often times missleading
df1['log Price'] = np.log(df1.loc[:,'Price'])

In [71]:
fig = df1[['log Price', 'PE']].iplot(
    subplots=True,
    shape=(2, 1),
    shared_xaxes=True,
    title="S&P500 time series with PE",
    asFigure=True
)

# Update layout
fig.update_layout(width=1000, height=600)  # Set figure size here

# Show the plot
iplot(fig)

/usr/local/lib/python3.10/dist-packages/cufflinks/plotlytools.py:117: FutureWarning:

DatetimeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.



The problem with multpl is that it only has the S&P500. What if we want to get P/E ratios for individual stocks? For that, we will extract the data from macrotrends

In [74]:
#the code is very similar the one from above, just some slight changes
def get_pe_from_url_macrotrends(ticker, company_name):
    url = f"https://www.macrotrends.net/stocks/charts/{ticker}/{company_name}/pe-ratio"

    #we need to pretend where are comming from a browser to scrape the data
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 '
            '(KHTML, like Gecko) Chrome/63.0.3239.108 Safari/537.36'
        )
    }

    response = requests.get(url, headers = headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')

    table = soup.find('table')
    for abbr in table.find_all('abbr'):
        abbr.decompose()

    # Initialize lists to store the extracted data
    dates = []
    values = []

    # Iterate over the table rows
    dates = []
    prices = []
    pe_ratios = []
    for row in table.find_all('tr')[2:]:  # Skip the header rows
            columns = row.find_all('td')
            date = columns[0].get_text(strip=True)
            price = columns[1].get_text(strip=True)
            pe_ratio = columns[3].get_text(strip=True)
            dates.append(date)
            prices.append(price)
            pe_ratios.append(pe_ratio)

    return [dates,prices,pe_ratios]

In [75]:
#example amazon pe ratio

data = get_pe_from_url_macrotrends('AMZN','amazon')

df = pd.DataFrame(data).T
df.columns = ['Date', 'Price', 'PE']
df = df.set_index('Date')

df.index = pd.to_datetime(df.index)
df = df.sort_index(ascending = False)

df.Price = pd.to_numeric(df.Price)
df.PE = pd.to_numeric(df.PE)

df.head()

,Price,PE
Date,,
2024-11-14,211.48,50.59
2024-06-30,193.25,46.23
2024-03-31,180.38,50.53
2023-12-31,151.94,52.39
2023-09-30,127.12,65.87


In [76]:
fig = df[['Price', 'PE']].iplot(
    subplots=True,
    shape=(2, 1),
    shared_xaxes=True,
    title="S&P500 time series with PE",
    asFigure=True
)

# Update layout
fig.update_layout(width=1000, height=600)  # Set figure size here

# Show the plot
iplot(fig)

/usr/local/lib/python3.10/dist-packages/cufflinks/plotlytools.py:117: FutureWarning:

DatetimeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.



In [77]:
#microsoft example
data = get_pe_from_url_macrotrends('msft','microsoft')

df = pd.DataFrame(data).T
df.columns = ['Date', 'Price', 'PE']
df = df.set_index('Date')

df.index = pd.to_datetime(df.index)
df = df.sort_index(ascending = False)

df.Price = pd.to_numeric(df.Price)
df.PE = pd.to_numeric(df.PE)
df['log Price'] = np.log(df['Price'])
df.head()

,Price,PE,log Price
Date,,,
2024-11-14,426.89,35.22,6.056526
2024-09-30,430.30,35.50,6.064483
2024-06-30,446.16,37.78,6.100678
2024-03-31,419.23,36.30,6.038420
2023-12-31,374.02,33.82,5.924309


In [79]:
fig = df[['log Price', 'PE']].iplot(
    subplots=True,
    shape=(2, 1),
    shared_xaxes=True,
    title="S&P500 time series with PE",
    asFigure=True
)

# Update layout
fig.update_layout(width=1000, height=600)  # Set figure size here

# Show the plot
iplot(fig)

/usr/local/lib/python3.10/dist-packages/cufflinks/plotlytools.py:117: FutureWarning:

DatetimeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.



In [80]:
#Johnson and Johnson
data = get_pe_from_url_macrotrends('JNJ','johnson-johnson')

df = pd.DataFrame(data).T
df.columns = ['Date', 'Price', 'PE']
df = df.set_index('Date')

df.index = pd.to_datetime(df.index)
df = df.sort_index(ascending = False)

df.Price = pd.to_numeric(df.Price)
df.PE = pd.to_numeric(df.PE)
df['log Price'] = np.log(df['Price'])
df.head()

,Price,PE,log Price
Date,,,
2024-11-14,151.87,21.98,5.023025
2024-09-30,162.06,23.45,5.087967
2024-06-30,145.06,9.06,4.977147
2024-03-31,155.72,9.71,5.048060
2023-12-31,153.13,11.09,5.031287


In [81]:
fig = df[['log Price', 'PE']].iplot(
    subplots=True,
    shape=(2, 1),
    shared_xaxes=True,
    title="S&P500 time series with PE",
    asFigure=True
)

# Update layout
fig.update_layout(width=1000, height=600)  # Set figure size here

# Show the plot
iplot(fig)

/usr/local/lib/python3.10/dist-packages/cufflinks/plotlytools.py:117: FutureWarning:

DatetimeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.

